In [20]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [21]:
datapath="/content/drive/MyDrive/Colab Notebooks/NLP/fake-news/train.csv"
testpath="/content/drive/MyDrive/Colab Notebooks/NLP/fake-news/test.csv"

In [22]:
df=pd.read_csv(datapath)
df=df.dropna()
df.reset_index(inplace=True)

In [23]:
y=df['label']
X=df.drop('label',axis=1)

In [24]:
X.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [25]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
import numpy as np

In [26]:
voc_size=10000

In [27]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
ps = PorterStemmer()
corpus=[]
for i in range(len(X)):
  senteence=re.sub('[^a-zA-Z]',' ',df['title'][i])
  senteence=senteence.lower()
  senteence=senteence.split()
  senteence=[ps.stem(word) for word in senteence if not word in stopwords.words('english')]
  sentence=' '.join(senteence)
  corpus.append(sentence)

In [29]:
t=[len(i) for i in corpus]

In [30]:
max(t)

299

In [31]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [32]:
sent_len=20

In [33]:
embedded_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_len)

In [34]:
embedding_feature=40
model=Sequential()
model.add(Embedding(voc_size,embedding_feature,input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            400000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
X_final=np.array(embedded_doc)
y_final=np.array(y)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [37]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 12ms/step - loss: 0.3204 - accuracy: 0.8467 - val_loss: 0.1840 - val_accuracy: 0.9210
Epoch 2/10
192/192 [==============================] - 2s 9ms/step - loss: 0.1202 - accuracy: 0.9536 - val_loss: 0.1853 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0733 - accuracy: 0.9742 - val_loss: 0.2207 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0469 - accuracy: 0.9846 - val_loss: 0.2825 - val_accuracy: 0.9196
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0308 - accuracy: 0.9907 - val_loss: 0.2739 - val_accuracy: 0.9160
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0171 - accuracy: 0.9943 - val_loss: 0.4330 - val_accuracy: 0.9171
Epoch 7/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0117 - accuracy: 0.9966 - val_loss: 0.3365 - val_accuracy: 0.9171
Ep

In [38]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
confusion_matrix(y_test,y_pred)

array([[3150,  269],
       [ 242, 2374]])

In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9153272576636289